In [1]:
import argparse
import numpy as np
import sys
from pymongo import MongoClient
import re
from __future__ import division
from sklearn.cluster import KMeans 
from numbers import Number
import pandas as pd
from pandas import DataFrame
import sys, codecs, numpy

# Prepare test, train and validation for cleaned/light processed/not-filtered data

In [250]:
original = pd.read_csv('dataset.csv', delimiter=',')
cleaned = pd.read_csv('cleaned_tweet.csv', delimiter=',')

In [251]:
empty = pd.isnull(cleaned).any(1).nonzero()[0]

In [76]:
len(empty)

61

In [77]:
original.SentimentText[empty]  #tweets that after filtration become nan

5602                                          #atishoo W1 4 
8871                                        &lt;3 alayellow 
11364                                         #shaundiviney 
547755                                         .....he's 19 
548410                                             arhhghgh 
557244                                        ACAAAAAAAABOU 
561786                                       Ai...to dodÃ³i 
581991                                          Acaaaaaabou 
610232                                            chillaxin 
627736       ChÃ³ng máº·t, nhá»©c Ä‘áº§u --&gt; ngá»§ váº­y 
640717     Dáº¡o nÃ y náº¯ng nÃ³ng, Ä‘Ã´i lÃºc mÃ¬nh nÃ³n...
652715                                       EW EW EW EW EW 
672799                           Ew ew ew ew ew ew ew ew... 
708101                                              F*ck it 
741877                                          He is...    
761998                                          hauuuuuuuss 
773559                  

In [252]:
original = original.drop(original.index[empty]).reset_index()

In [253]:
original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1578564 entries, 0 to 1578563
Data columns (total 5 columns):
index              1578564 non-null int64
﻿ItemID            1578564 non-null int64
Sentiment          1578564 non-null int64
SentimentSource    1578564 non-null object
SentimentText      1578564 non-null object
dtypes: int64(3), object(2)
memory usage: 60.2+ MB


In [254]:
cleaned = cleaned.dropna().reset_index()
cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1578564 entries, 0 to 1578563
Data columns (total 3 columns):
index        1578564 non-null int64
sentiment    1578564 non-null int64
text         1578564 non-null object
dtypes: int64(2), object(1)
memory usage: 36.1+ MB


In [255]:
cleaned.text.ix[5655]

'follow friday exclamation user nicest girl ever'

In [256]:
original.SentimentText.ix[5655]

'#FOLLOWFRIDAY! ----&gt; @LeezeArray &lt;---- nicest girl ever '

In [257]:
positive_tweets = cleaned[cleaned.sentiment == 1]
negative_tweets = cleaned[cleaned.sentiment == 0]
len(positive_tweets), len(negative_tweets)

(790164, 788400)

In [258]:
positive_tweets = positive_tweets[:788400]
negative_tweets = negative_tweets[:788400]
print len(positive_tweets), len(negative_tweets)

788400 788400


In [259]:
base = pd.concat([positive_tweets, negative_tweets], axis=0, ignore_index=True)
print base.columns

Index([u'index', u'sentiment', u'text'], dtype='object')


In [260]:
l = base.index
cleaned = cleaned.ix[l].reset_index()
original = original.ix[l].reset_index()

In [262]:
from sklearn.cross_validation import train_test_split
X_train_val, X_test, y_train_val, y_test = train_test_split(base['text'], base['sentiment'], test_size=0.1)
X_train, X_validation, y_train, y_validation = train_test_split(X_train_val, y_train_val, test_size=0.1)
X_train.shape, X_validation.shape, X_test.shape

((1277208,), (141912,), (157680,))

In [263]:
train = X_train.index
test = X_test.index
val = X_validation.index

In [271]:
X_train = cleaned.text.ix[train].values
X_test = cleaned.text.ix[test].values
X_validation = cleaned.text.ix[val].values

y_train = cleaned.sentiment.ix[train].values
y_test = cleaned.sentiment.ix[test].values
y_validation = cleaned.sentiment.ix[val].values

XX_train = original.SentimentText.ix[train].values
XX_test  = original.SentimentText.ix[test].values
XX_validation  = original.SentimentText[val].values

yy_train = original.Sentiment[train]
yy_test = original.Sentiment[test]
yy_validation = original.Sentiment[val]

In [272]:
X_test[34] , '\n', XX_test[34]

('duty with abi', '\n', 'on duty with abi ')

In [273]:
X_train[34] , '\n', XX_train[34]

('user dang were playing and was all night taking care sick baby and missed exclamation',
 '\n',
 '@jordanknight Dang we were playing and I was up all night taking care of a sick baby and missed it! ')

In [274]:
X_validation[34] , '\n', XX_validation[34]

('user lot mind question always exclamation',
 '\n',
 '@BlowhornOz A lot on my mind? Always! ')

In [275]:
df = pd.DataFrame({'text': X_train, 'sentiment': y_train})
print(len(df))
df.to_csv("archive/cleaned_tweet_train.csv", index=False)

1277208


In [276]:
df = pd.DataFrame({'text': X_test, 'sentiment': y_test})
print(len(df))
df.to_csv("archive/cleaned_tweet_test.csv", index=False)

157680


In [277]:
df = pd.DataFrame({'text': X_validation, 'sentiment': y_validation})
print(len(df))
df.to_csv("archive/cleaned_tweet_validation.csv", index=False)

141912


In [278]:
df = pd.DataFrame({'text': XX_train, 'sentiment': yy_train})
df.to_csv("archive/not_filtered_train.csv", index=False)

df = pd.DataFrame({'text': XX_test, 'sentiment': yy_test})
df.to_csv("archive/not_filtered_test.csv", index=False)

df = pd.DataFrame({'text': XX_validation, 'sentiment': yy_validation})
df.to_csv("archive/not_filtered_validation.csv", index=False)

In [25]:
class autovivify_list(dict):
        '''Pickleable class to replicate the functionality of collections.defaultdict'''
        def __missing__(self, key):
                value = self[key] = []
                return value

        def __add__(self, x):
                '''Override addition for numeric types when self is empty'''
                if not self and isinstance(x, Number):
                        return x
                raise ValueError

        def __sub__(self, x):
                '''Also provide subtraction method'''
                if not self and isinstance(x, Number):
                        return -1 * x
                raise ValueError

In [26]:
def build_word_vector_matrix(vector_file):
        '''Read a GloVe array from sys.argv[1] and return its vectors and labels as arrays'''
        numpy_arrays = []
        labels_array = []
        with codecs.open(vector_file, 'r', 'utf-8') as f:
            for c, r in enumerate(f):
                sr = r.split()
                labels_array.append(sr[0])
                vec = numpy.array([float(i) for i in sr[1:]])
                numpy_arrays.append(vec)
    

                    #if c == n_words:
                    #        return numpy.array( numpy_arrays ), labels_array

        return numpy.array( numpy_arrays ), labels_array

In [27]:
input_vector_file = 'glove.6B.200d.txt' 
df, labels_array  = build_word_vector_matrix(input_vector_file)

In [28]:
df.shape

(400000, 200)

In [29]:
def generate(df, lables):
    words = lables
    vocab = {w: idx for idx, w in enumerate(words)}
    ivocab = {idx: w for idx, w in enumerate(words)}
    
    W =  df
    # normalize each word vector to unit variance
    W_norm = np.zeros(W.shape)
    d = (np.sum(W ** 2, 1) ** (0.5))
    W_norm = (W.T / d).T
    return (W_norm, vocab, ivocab)

In [30]:
W, vocab, ivocab = generate(df, labels_array)

In [31]:
def get_vec(W, vocab, ivocab, input_term, fail):
    vecs = []
    for idx, term in enumerate(input_term.split(' ')):
        if term in vocab:
            vecs.append(W[vocab[term], :])
        else:
            fail = fail + 1

    size = len(vecs)
    if size == 0:
        vec_result = np.zeros((200,))
    else:
        vec_result = np.array(vecs).sum(axis = 0)/size
    vec_norm = np.zeros(vec_result.shape)
    d = (np.sum(vec_result ** 2,) ** (0.5))
    vec_norm = (vec_result.T / d).T
    return np.array(vec_norm)

In [2]:
import datetime
import pickle

In [33]:
def vec_calc(data):
    fail = 0
    t = datetime.datetime.now()
    vectors = []
    for tweet in data:
        v = get_vec(W, vocab, ivocab, tweet,fail)
        vectors.append(v)
    print datetime.datetime.now() - t
    print 'fail to find ', fail
    return vectors

In [60]:
cl_db  =  pd.read_csv('archive/clean/cleaned_tweet_train.csv', delimiter=',')
X_train = cl_db.text.values
X_train.shape

(1277208,)

In [61]:
cl_db_test  =  pd.read_csv('archive/clean/cleaned_tweet_test.csv', delimiter=',')
X_test = cl_db_test.text.values
X_test.shape

(157680,)

In [63]:
cl_db_val  =  pd.read_csv('archive/clean/cleaned_tweet_validation.csv', delimiter=',')
X_validation = cl_db_val.text.values
X_validation.shape

(141912,)

In [77]:
vectors = vec_calc(X_train)
vectors = np.array(vectors)
print vectors.shape
f = open( "archive/clean_gl_tw_tr_vec.pickle", "wb" )
pickle.dump(vectors, f)
f.close()

v_test = vec_calc(X_test)
v_test = np.array(v_test)
print v_test.shape
f = open( "archive/clean_gl_tw_tes_vec.pickle", "wb" )
pickle.dump(v_test, f)
f.close()

v_val = vec_calc(X_validation)
v_val = np.array(v_val)
print v_val.shape
f = open( "archive/clean_gl_tw_val_vec.pickle", "wb" )
pickle.dump(v_val, f)
f.close()

0:01:13.513994
fail to find  0
(1277208, 200)
0:00:28.115516
fail to find  0
(157680, 200)
0:00:10.955867
fail to find  0
(141912, 200)


In [22]:
cl_db  =  pd.read_csv('archive/lf/lf_tweet_train.csv', delimiter=',')
XX_train = cl_db.text.values
print XX_train.shape

cl_db_test  =  pd.read_csv('archive/lf/lf_tweet_test.csv', delimiter=',')
XX_test = cl_db_test.text.values
print XX_test.shape


cl_db_val  =  pd.read_csv('archive/lf/lf_tweet_val.csv', delimiter=',')
XX_validation = cl_db_val.text.values
print XX_validation.shape

(1277208,)
(157680,)
(141912,)


In [81]:
vectors = vec_calc(XX_train)
vectors = np.array(vectors)
print vectors.shape
f = open( "archive/lf_gl_tw_tr_vec.pickle", "wb" )
pickle.dump(vectors, f)
f.close()

v_test = vec_calc(XX_test)
v_test = np.array(v_test)
print v_test.shape
f = open( "archive/lf_gl_tw_tes_vec.pickle", "wb" )
pickle.dump(v_test, f)
f.close()

v_val = vec_calc(X_validation)
v_val = np.array(v_val)
print v_val.shape
f = open( "archive/lf_gl_tw_val_vec.pickle", "wb" )
pickle.dump(v_val, f)
f.close()

0:01:46.985670
fail to find  0
(1277208, 200)
0:00:20.665048
fail to find  0
(157680, 200)
0:00:08.173685
fail to find  0
(141912, 200)


In [296]:
vectors = vec_calc(X_train)
vectors = np.array(vectors)
print vectors.shape
f = open( "archive/clean_gl_wiki_tr_vec.pickle", "wb" )
pickle.dump(vectors, f)
f.close()

0:01:14.934657
fail to find  0
(1277208, 200)


In [297]:
v = pickle.load(open('archive/clean_gl_wiki_tr_vec.pickle', 'rb'))
v[199]==vectors[199]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [298]:
v_test = vec_calc(X_test)
v_test = np.array(v_test)
print v_test.shape
f = open( "archive/clean_gl_wiki_tes_vec.pickle", "wb" )
pickle.dump(v_test, f)
f.close()

0:00:14.341741
fail to find  0
(157680, 200)


In [299]:
v_val = vec_calc(X_validation)
v_val = np.array(v_val)
print v_val.shape
f = open( "archive/clean_gl_wiki_val_vec.pickle", "wb" )
pickle.dump(v_val, f)
f.close()

0:00:08.338756
fail to find  0
(141912, 200)


In [300]:
vectors = vec_calc(XX_train)
vectors = np.array(vectors)
print vectors.shape
f = open( "archive/lf_gl_wiki_tr_vec.pickle", "wb" )
pickle.dump(vectors, f)
f.close()

0:01:17.200324
fail to find  0
(1277208,)


In [36]:
f = open( "archive/lf_gl_wiki_tr_vec.pickle", "wb" )
pickle.dump(vectors, f)
f.close()

In [34]:
vectors = vec_calc(XX_train)
vectors = np.array(vectors)
print vectors.shape

0:01:22.486443
fail to find  0
(1277208, 200)


In [37]:
v_test = vec_calc(XX_test)
v_test = np.array(v_test)
print v_test.shape
f = open( "archive/lf_gl_wiki_tes_vec.pickle", "wb" )
pickle.dump(v_test, f)
f.close()

0:00:18.349719
fail to find  0
(157680, 200)


In [302]:
v_val = vec_calc(X_validation)
v_val = np.array(v_val)
print v_val.shape
f = open( "archive/lf_gl_wiki_val_vec.pickle", "wb" )
pickle.dump(v_val, f)
f.close()

0:00:08.405389
fail to find  0
(141912, 200)


In [279]:
dset = pd.read_csv('archive/not_filtered_test.csv', delimiter=',')

In [280]:
dset.ix[2]

sentiment                     0
text         no ISP on Monday. 
Name: 2, dtype: object

In [281]:
dset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157680 entries, 0 to 157679
Data columns (total 2 columns):
sentiment    157680 non-null int64
text         157680 non-null object
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [282]:
l = 157680
XXX_test = []
for i in xrange(l):
    t = dset.text.ix[i] 
    text = light_process(t)
    XXX_test.append(text)

In [283]:
XXX_test[2]

'no isp on monday '

In [284]:
df = pd.DataFrame({'text': XXX_test, 'sentiment': dset.sentiment})
print(len(df))
df.to_csv("archive/lf_tweet_test.csv", index=False)

157680


In [287]:
dset = pd.read_csv('archive/not_filtered_train.csv', delimiter=',')
dset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1277208 entries, 0 to 1277207
Data columns (total 2 columns):
sentiment    1277208 non-null int64
text         1277208 non-null object
dtypes: int64(1), object(1)
memory usage: 19.5+ MB


In [288]:
l = 1277208
XXX_train = []
for i in xrange(l):
    t = dset.text.ix[i] 
    text = light_process(t)
    XXX_train.append(text)

In [289]:
df = pd.DataFrame({'text': XXX_train, 'sentiment': dset.sentiment})
df.to_csv("archive/lf_tweet_train.csv", index=False)

In [290]:
dset = pd.read_csv('archive/not_filtered_validation.csv', delimiter=',')
dset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141912 entries, 0 to 141911
Data columns (total 2 columns):
sentiment    141912 non-null int64
text         141912 non-null object
dtypes: int64(1), object(1)
memory usage: 2.2+ MB


In [291]:
l = 141912
XXX_train = []
for i in xrange(l):
    t = dset.text.ix[i] 
    text = light_process(t)
    XXX_train.append(text)

In [292]:
df = pd.DataFrame({'text': XXX_train, 'sentiment': dset.sentiment})
df.to_csv("archive/lf_tweet_val.csv", index=False)

In [4]:
cwiki_train = pickle.load(open('archive/glove_wiki/clean/clean_gl_wiki_tr_vec.pickle', 'rb'))

In [5]:
cwiki_tr_dset = pd.read_csv('archive/clean/cleaned_tweet_train.csv', delimiter=',')

In [6]:
cwiki_tr_lab = cwiki_tr_dset.sentiment.values

In [7]:
cwiki_tr_lab.shape, cwiki_train.shape

((1277208,), (1277208, 200))

In [3]:
train_size = 80000
test_size = 10000

In [11]:
from sklearn import svm
clf = svm.SVC()
clf.fit(cwiki_train[:train_size],cwiki_tr_lab[:train_size]) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [13]:
cwiki_test = pickle.load(open('archive/glove_wiki/clean/clean_gl_wiki_tes_vec.pickle', 'rb'))

In [15]:
cwiki_tes_dset = pd.read_csv('archive/clean/cleaned_tweet_test.csv', delimiter=',')
cwiki_test_lab = cwiki_tes_dset.sentiment.values
cwiki_test_lab.shape, cwiki_test.shape

((157680,), (157680, 200))

In [17]:
def classify(tweet):
    return clf.predict(tweet)

In [19]:
labels = []

for vec in cwiki_test[:test_size]:
    vec = vec.reshape(1,-1)
    label = classify(vec)
    labels.append(label)

In [20]:
counter = 0
for i, l in enumerate(cwiki_test_lab[:test_size]):
    if l == labels[i]:
        counter +=1

In [21]:
print counter/len(labels)

0.7115


In [3]:
ctw_train = pickle.load(open('archive/glove_twitter/clean/clean_gl_tw_tr_vec.pickle', 'rb'))
ctw_train.shape

(1277208, 200)

In [8]:
ctw_tr_dset = pd.read_csv('archive/clean/cleaned_tweet_train.csv', delimiter=',')
ctw_tr_lab = ctw_tr_dset.sentiment.values

In [9]:
ctw_tr_lab.shape

(1277208,)

In [13]:
from sklearn import svm
clf = svm.SVC()
clf.fit(ctw_train[:train_size],ctw_tr_lab[:train_size]) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [16]:
ctw_test = pickle.load(open('archive/glove_twitter/clean/clean_gl_tw_tes_vec.pickle', 'rb'))
ctw_test_dset = pd.read_csv('archive/clean/cleaned_tweet_test.csv', delimiter=',')
ctw_test_lab = ctw_test_dset.sentiment.values
ctw_test_lab.shape, ctw_test.shape

((157680,), (157680, 200))

In [20]:
labels = []

for vec in ctw_test[:test_size]:
    vec = vec.reshape(1,-1)
    label = classify(vec)
    labels.append(label)
    
counter = 0
for i, l in enumerate(ctw_test_lab[:test_size]):
    if l == labels[i]:
        counter +=1
print counter/len(labels)

0.7342


In [5]:
lft_train = pickle.load(open('archive/glove_twitter/lf/lf_gl_tw_tr_vec.pickle', 'rb'))
lft_train.shape

(1277208, 200)

In [6]:
lft_tr_dset = pd.read_csv('archive/lf/lf_tweet_train.csv', delimiter=',')
lft_tr_lab = lft_tr_dset.sentiment.values

In [7]:
lft_tr_lab.shape

(1277208,)

In [8]:
n =  np.where(np.isnan(lft_train[:train_size]))

In [9]:
n = np.array(n)
n = np.unique(n[0,:])

In [10]:
n

array([  855, 10148, 13625, 19709, 25558, 33375, 36026, 39831, 44481,
       47679, 50059, 58155, 59421, 62027, 62103, 62424, 74119, 74775,
       74926, 78275])

In [11]:
lft_train[n,:] = np.zeros((200,))

In [12]:
lft_train[:train_size].shape, lft_tr_lab[:train_size].shape

((80000, 200), (80000,))

In [13]:
from sklearn import svm
clf = svm.SVC()
clf.fit(lft_train[:train_size],lft_tr_lab[:train_size])

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [47]:
lft_test = pickle.load(open('archive/glove_twitter/lf/lf_gl_tw_tes_vec.pickle', 'rb'))
lft_test_dset = pd.read_csv('archive/lf/lf_tweet_test.csv', delimiter=',')
lft_test_lab = lft_test_dset.sentiment.values
lft_test_lab.shape, lft_test.shape

((157680,), (157680, 200))

In [48]:
lft_test[:test_size].shape

(10000, 200)

In [49]:
n =  np.where(np.isnan(lft_test[:test_size]))
n = np.array(n)
n = np.unique(n[0,:])
print n.shape

(2,)


In [50]:
lft_test[n,:] = np.zeros((200,))

In [51]:
lft_test.shape

(157680, 200)

In [52]:
labels = []

for vec in lft_test[:test_size]:
    vec = vec.reshape(1,-1)
    label = classify(vec)
    labels.append(label)

In [53]:
counter = 0
for i, l in enumerate(lft_test_lab[:test_size]):
    if l == labels[i]:
        counter +=1
print counter/len(labels)

0.7311


In [55]:
lfw_train = pickle.load(open('archive/lf_gl_wiki_tr_vec.pickle', 'rb'))
lfw_train.shape 

(1277208, 200)

In [56]:
lfw_tr_dset = pd.read_csv('archive/lf/lf_tweet_train.csv', delimiter=',')
lfw_tr_lab = lfw_tr_dset.sentiment.values
lfw_tr_lab.shape

(1277208,)

In [57]:
n =  np.where(np.isnan(lfw_train[:train_size]))
n = np.array(n)
n = np.unique(n[0,:])
print n.shape

(39,)


In [58]:
lfw_train[n,:] = np.zeros((200,))

In [59]:
lfw_train[:train_size].shape,lfw_tr_lab[:train_size].shape

((80000, 200), (80000,))

In [61]:
clf = svm.SVC()
clf.fit(lfw_train[:train_size],lfw_tr_lab[:train_size])

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [63]:
lfw_test = pickle.load(open('archive/lf_gl_wiki_tes_vec.pickle', 'rb'))
lfw_test_dset = pd.read_csv('archive/lf/lf_tweet_test.csv', delimiter=',')
lfw_test_lab = lfw_test_dset.sentiment.values
lfw_test_lab.shape, lfw_test.shape

((157680,), (157680, 200))

In [64]:
n =  np.where(np.isnan(lfw_test[:test_size]))
n = np.array(n)
n = np.unique(n[0,:])
print n.shape

(7,)


In [65]:
lfw_test[n,:] = np.zeros((200,))

In [66]:
labels = []

for vec in lfw_test[:test_size]:
    vec = vec.reshape(1,-1)
    label = classify(vec)
    labels.append(label)

In [67]:
counter = 0
for i, l in enumerate(lfw_test_lab[:test_size]):
    if l == labels[i]:
        counter +=1
print counter/len(labels)

0.7051


In [509]:
wtf = np.array(wtf)
wtf = np.unique(wtf)
wtf.shape

(5761,)

In [510]:
wtf[:100]

array(['aabangan', 'aaeeaa', 'aafekm', 'aagenealogy',
       'aahahahahahahahahahahahahahahahah', 'aahahahha', 'aahmhm',
       'aaiihh', 'aalinakirche', 'aaliyahlove', 'aamaustin', 'aandhi',
       'aanswwe', 'aaooww', 'aargghh', 'aarghh', 'aarrgghh', 'aarrgh',
       'aawurl', 'aawwhh', 'abangmu', 'abcfamily',
       'abc\xc3\xaf\xc2\xbf\xc2\xbds', 'abduzeedo', 'abduzzeedo',
       'abfahrt', 'abigaillyy', 'abondigas', 'abrazos', 'abseloutly',
       'aburriendomee', 'ab\xc3\x83\xc2\xaeme', 'acadeaua',
       'accesshollywood', 'aceburpeeshow', 'aceisancient', 'aceleasi',
       'acenetc', 'aceybear', 'aclientsplatform', 'acrisoare',
       'acrylicana', 'actionfigure', 'activisionsucks', 'actofgreen',
       'acupunc', 'adamcheeserosie', 'adamgazm', 'adamisarockstar',
       'adamlambert', 'adamliveson', 'addictivetvshows', 'adiabatman',
       'adictaa', 'adinarayanap', 'adt\xc3\xaf\xc2\xbf\xc2\xbd',
       'adultbaby', 'adultswim', 'adverblog', 'adweebs', 'afazenda',
       'affil

aagenealogy
activisionsucks
affright
angelically
armhole
bondman
butchano
caleche
carious
cassadinkle
castigatorul
chalkpit
chigoe
chilblain
churrcchh
cocksucker
confuzzled
crackalackin
cyberhallelujah
danceordiewithavengeance
dayquil
deevvoo
duvelisdendrank
ecomonday
ferrarii
fichu
frappachins
frappucino
greenflys
gtretweet
halfwitt
hugglesxx
imisscath
inaninyonga
incise
invitanï¿½
laceylynnwilliams
ladyloveslondonatgmaildotcom
madadapa
masterbating
mayblowyourmind
membra
memoribillia
metaphysis
motorycle
mozmemories
multumesc
mygyver
myspizzle
octopods
officialbowwow
pearshaped
pelforth
placethere
playfellow
pneumonoultramicroscopicsilicovolcanoconiosis
pubes
puttee
reeyasabellina
skyblue


KeyboardInterrupt: 

In [537]:
yes[:100]

['aba',
 'age',
 'aha',
 'haha',
 'haha',
 'haha',
 'haha',
 'haha',
 'haha',
 'haha',
 'aha',
 'yah',
 'love',
 'ama',
 'tin',
 'and',
 'aba',
 'abc',
 'family',
 'abc',
 'zee',
 'zee',
 'abigail',
 'dig',
 'abra',
 'lout',
 'rien',
 'dome',
 'ade',
 'access',
 'holly',
 'wood',
 'ace',
 'burp',
 'how',
 'ace',
 'ancient',
 'ace',
 'leas',
 'ace',
 'net',
 'ace',
 'bear',
 'lien',
 'splat',
 'form',
 'acris',
 'oar',
 'acrylic',
 'ana',
 'action',
 'figure',
 'vision',
 'suck',
 'act',
 'green',
 'acu',
 'pun',
 'adam',
 'cheese',
 'adam',
 'adam',
 'adam',
 'lambert',
 'adam',
 'lives',
 'addictive',
 'show',
 'aba',
 'man',
 'ara',
 'yana',
 'adult',
 'baby',
 'adult',
 'swim',
 'adverb',
 'log',
 'wee',
 'faze',
 'icon',
 'after',
 'prom',
 'after',
 'sun',
 'after',
 'the',
 'fact',
 'agar',
 'age',
 'play',
 'agree',
 'gre',
 'wee',
 'aha',
 'aha',
 'haha',
 'aha',
 'haha',
 'aha']

In [491]:
wtf.shape

(5833,)

In [492]:
wtf[:100]

array(['0116', '0546', '0631', '09011', '108370987500', '11327',
       '1234567890', '12800', '13341015518', '1337357', '13850', '14517',
       '1586789', '18312', '184948', '1898553', '19627', '2013212491',
       '2174', '22850', '2669', '2790', '2807', '3175228858', '32509',
       '361733', '4291', '4357', '43829', '4434156058', '4650', '4980',
       '5015', '5183', '5278020', '5319', '53344', '533939',
       '5346543212121214572', '5610', '5678', '6034385620', '61423682984',
       '6156183900', '6173', '621621621', '6280', '6380', '6597881990',
       '6958', '708836', '723897', '74892', '7837', '7897389472', '8193',
       '8199', '8330', '8537', '85455', '864290', '8675309',
       '89374789329478', '90028', '9157', '9203427290', '923400', '92683',
       '9401', '9435', '9496808819', '9553', 'aabangan', 'aaeeaa',
       'aafekm', 'aagenealogy', 'aahahahahahahahahahahahahahahahah',
       'aahahahha', 'aahmhm', 'aaiihh', 'aalinakirche', 'aaliyahlove',
       'aamaustin', 'a

In [23]:
def InitializeWords():
    wordlist = 'dic.txt' # A file containing common english words
    content = None
    with open(wordlist) as f:
        content = f.readlines()
    return [word.rstrip('\n') for word in content]


def ParseWord(term, wordlist):
    words = []
    # Remove hashtag, split by dash
    word = FindWord(term, wordlist)    
    while word != None and len(term) > 0:
        words += [word]            
        if len(term) == len(word): # Special case for when eating rest of word
            break
        term = term[len(word):]
        word = FindWord(term, wordlist)
    return " ".join(words)


def FindWord(token, wordlist):
    i = len(token) + 1
    while i > 1:
        i -= 1
        if token[:i] in wordlist:
            return token[:i]
    return None 


In [503]:
wordlist = InitializeWords()
for word in wtf[100:200]:
    w = ParseWord(word, wordlist)
    print w

abra z os
ab se lout ly
ab ur rien dome e
ab
ac ade au a
access holly wood
ace burp e es how
ace is ancient
ace leas i
ace net c
ace y bear
ac lien t splat form
acris oar e
acrylic ana
action figure
acti vision suck s
act of green
acu pun c
adam cheese ro si e
adam ga z m
adam is ar o c
adam lambert
adam lives on
addictive t v show s
ad i aba t man
ad i c ta a
ad in ara yana p
ad t
adult baby
adult swim
adverb log
ad wee b s
a faze nd a
a ff il icon il
affright
after prom
after sun
after the fact
agar ra r
age play
agree gre e
a g wee
aha ha
aha haha ha
aha haha ha
aha haha haha
aha haha haha a
aha haha haha
aha haha haha ha
aha haha haha haha ha
aha ha
ah ha aha ha
ah
ah mad in
ah mad in ed sc had
ah o o va
ah su has u ha
ah
ah
a i
air con d
air condition
air fr ace
air france
air master
air mood
air tel s
a is ha za m
a i ye rh
a ja ed and ridge
a ja ja ja ja ja
a
a kh i r ny a
a kj a va
a komi sm o
a
a
a kw tj wk l
ala nc ar r
ala nis sarcastic
alba n na ch
ale nina
ale rg i c
alex 

In [ ]:
import json

with open('tw-abr-dict.json') as data_file:    
    data = json.load(data_file)

In [421]:
wtf.shape

(6239,)

In [422]:
wtf[:100]

array(['0546', '0631', '09011', '093m3', '0o0o0oh', '100gb', '100miles',
       '100miletraining', '100ms', '100pushups', '100times', '100v',
       '10celcius', '10h30m', '10mins', '10yds', '10yearsofenema', '10yrs',
       '11327', '120gb', '120k', '12800', '1280x800', '128gbs', '128pc',
       '12\xc3\x82\xc2\xb0c', '130am', '13341015518', '1337357', '1337th',
       '13850', '13lbs', '13musiconline', '13yrold', '140charfail',
       '140conf', '140smiles', '140tc', '1500k', '150mb', '1580px',
       '1586789', '15hrs', '15yrs', '160gb', '16mbps', '16thru21', '16wks',
       '18312', '184948', '1898553', '18h30m', '18hrs', '18yr', '1900txts',
       '195bucks', '19627', '1stds10', '1weikert', '200ad', '2013212491',
       '20gb', '20lbs', '20mins', '20x5', '20yrs', '2174', '21yr', '22850',
       '232\xc3\xaf\xc2\xbf\xc2\xbd', '234am', '23mins', '250pp',
       '256kbps', '25mins', '260k', '2669', '2790', '27hrs', '2807',
       '28kbps', '2cos2x', '2getha', '2junxion', '2morra', '2

In [327]:
wtf.shape #without spliting hashtags

(13890,)

In [345]:
wtf[:100]

['websiite',
 'wallpaperdont',
 'quotsighquot',
 'quotlord',
 'ahahah',
 'mollybut',
 'unfollowdiddy',
 'quotfor',
 'namequot',
 'namegood',
 'whyquot',
 'snowhite',
 'asracist',
 'russfail',
 'squarespace',
 'hahaill',
 'robotpickuplines',
 'thecsperspectiveuser',
 'isembeck',
 'hadn',
 'blaarhg',
 'raingodd',
 'quotthinnerquot',
 'alliee',
 'arghghgghoh',
 'shakespearesaturday',
 'quotwatching',
 'elephantsquot',
 'puttee',
 'espesh',
 'followfriday',
 'hehehedead',
 'iranelection',
 'voc\xc3\xaf\xc2\xbf\xc2\xbds',
 'quottwittandoquot',
 'a\xc3\xaf\xc2\xbf\xc2\xbd',
 'hahahahah',
 'prreeyty',
 '\xc3\x82\xc2\xb4re',
 'solutionproject',
 'babymommaa',
 'eitherfail',
 'qualifyingrace',
 'quotyou',
 'youquotand',
 'girliegirl',
 'cutegirl',
 'funkygirl',
 'followfriday',
 'woteva',
 'sherdinator',
 '\xc3\xaf\xc2\xbf\xc2\xbd',
 'textswtf',
 'hmmlooking',
 'itdamnkids',
 'quotsettingsquot',
 'keybindings',
 'thingsihate',
 'forevaahh',
 'followfridayprrs',
 'babyintheoven',
 'calendimaggio

In [319]:
wtf.shape #old. Do not touch! 

(26059,)

In [328]:
tweets = np.array(tweets)

In [329]:
tweets.shape

(80000, 50)

In [330]:
from sklearn import svm
clf = svm.SVC()
clf.fit(tweets,y_train) 

def classify(tweet):
    return clf.predict(tweet)

In [334]:
labels = []

for tweet in X_test:
    tweet = processTweet(tweet)
    vec = get_vec(W, vocab, ivocab,tweet).reshape(1,-1)
    if vec.shape != (1,50):
        vec = np.zeros((1,50))
    label = classify(vec)
    labels.append(label)

In [335]:
labels = np.array(labels)
labels.shape

(20000, 1)

In [336]:
counter = 0
for i, l in enumerate(y_test):
    if l == labels[i]:
        counter +=1

In [337]:
print counter/len(labels)

0.6686


In [338]:
# define all useful functions
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]
        
# define yout train and accuracy functions first         
def train_nn(X_train, y_train, train_f, accuracy_f, num_epochs = 20, batch_size = 100, verbose = False):
    for epoch in range(num_epochs):
        # In each epoch, we do a full pass over the training data:
        train_err = 0
        train_acc = 0
        train_batches = 0
        start_time = time.time()
        for batch in iterate_minibatches(X_train, y_train,batch_size):
            inputs, targets = batch
            targets = targets.reshape(batch_size,1)
            train_err_batch, train_acc_batch = train_f(inputs, targets)
            train_err += train_err_batch
            train_acc += train_acc_batch 
            train_batches += 1

#         # And a full pass over the validation data:
#         val_acc = 0
#         val_batches = 0
#         for batch in iterate_minibatches(X_val, y_val, batch_size):
#             inputs, targets = batch
#             val_acc += accuracy_f(inputs, targets)
#             val_batches += 1
        
        if verbose:
            print("Epoch {} of {} took {:.3f}s".format(
                epoch + 1, num_epochs, time.time() - start_time))

            print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
            print("  train accuracy:\t\t{:.2f} %".format(
                train_acc / train_batches * 100))
#             print("  validation accuracy:\t\t{:.2f} %".format(
#                 val_acc / val_batches * 100))
        
    train_accuracy = train_acc / train_batches * 100
#     validation_accuracy = val_acc / val_batches * 100
    
    validation_accuracy = 0
    
    return train_accuracy, validation_accuracy


def test_nn(X_test, y_test, accuracy_f, batch_size = 100, verbose = False):
    test_acc = 0
    test_batches = 0
    
    for batch in iterate_minibatches(X_test, y_test, batch_size):
        inputs, targets = batch
        targets = targets.reshape(batch_size,1)
        acc = accuracy_f(inputs, targets)
        test_acc += acc
        test_batches += 1
        
    test_accuracy = test_acc / test_batches * 100
    
    return test_accuracy

def conv_nn(input_X, input_shape, dropout = False):
    net = lasagne.layers.InputLayer(shape = input_shape, input_var=input_X)
    
    nets = []
    #hidden
    for filter_size in filter_sizes:
        temp_net = lasagne.layers.Conv1DLayer(net, num_filters=num_filters, filter_size=filter_size)
        temp_net = lasagne.layers.MaxPool1DLayer(net, pool_size=2)
        
        nets.append(temp_net)
        
    # concat layers for different filter sizes     
    net = lasagne.layers.concat(nets) 
    
    net = lasagne.layers.DenseLayer(net, num_units = 50,
                                         nonlinearity = lasagne.nonlinearities.rectify)
    # output
    net = lasagne.layers.DenseLayer(net, num_units = 1,
                                         nonlinearity = lasagne.nonlinearities.sigmoid,
                                         name='output')
    return net

In [339]:
import lasagne
import theano
import theano.tensor as T

n_dim = 50
num_filters = 50
filter_sizes = [3,4,5]
input_shape = [None, 1, n_dim]

input_X = T.tensor3('X', dtype="float32")
input_y = T.matrix('y', dtype="int32")
target_y = T.matrix("target Y",dtype='int32')


train_v = X_train
test_v = X_test

In [340]:
# training parameters
num_epochs = 400   #amount of passes through the data
batch_size = 400   #number of samples processed at each function call

net = conv_nn(input_X, input_shape=input_shape)

# basic training loop 

#network prediction (theano-transformation)
y_predicted = lasagne.layers.get_output(net)

#all network weights (shared variables)
all_weights = lasagne.layers.get_all_params(net, trainable = True)

#binary crossentropy as loss function
loss = lasagne.objectives.binary_crossentropy(y_predicted, target_y).mean()

#prediction accuracy
accuracy = lasagne.objectives.binary_accuracy(y_predicted, target_y).mean()

#gradient computation + weights update
updates = lasagne.updates.rmsprop(loss, all_weights, learning_rate=0.001)

#function that computes loss and updates weights
train = theano.function([input_X, target_y],[loss, accuracy], updates = updates, allow_input_downcast=True)

#function that just computes accuracy
accuracy = theano.function([input_X,target_y], accuracy, allow_input_downcast=True)

In [ ]:
tweets = []

for t in X_train[:100]:
    print t
    t = processTweet(t)
    print t
    v = get_vec(W, vocab, ivocab, t).reshape(1,-1)
    if v.shape != (1,50):
        v = np.zeros((1,50))
    tweets.append(np.array(v))

In [ ]:
tweets = np.array(tweets)
tweets.shape

In [ ]:
test_tweet = []

for t in X_test:
    t = processTweet(t)
    v = get_vec(W, vocab, ivocab, t).reshape(1,-1)
    if v.shape != (1,50):
        v = np.zeros((1,50))
    test_tweet.append(np.array(v))

test_tweet = np.array(test_tweet)
test_tweet.shape, y_test.shape

In [ ]:
train_accuracy, validation_accuracy = train_nn(tweets, y_train, train, accuracy,
                                               num_epochs = num_epochs, 
                                               batch_size = batch_size, verbose = True)

test_accuracy = test_nn(test_tweet, y_test, accuracy, batch_size = 400, verbose = False)

print("conv nn ")
print("  train accuracy:", train_accuracy)
# print("  validation accuracy:", validation_accuracy)
print("  test accuracy:", test_accuracy)
print("____________________________________________")